<h1>Download One Specific File</h1>

In [1]:
import os
import requests
import numpy as np

In [7]:
# download xlsx file, ppp-data-dictionary

dictionary_url = "https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/aab8e9f9-36d1-42e1-b3ba-e59c79f1d7f0/download/ppp-data-dictionary.xlsx"
resp = requests.get(dictionary_url)
with open('ppp-data-dictionary.xlsx', 'wb') as output:
    output.write(resp.content)

In [8]:
# download one csv file, 

CSV_URL = 'https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/7371d1ca-57e8-4efb-9993-780cea937449/download/public_up_to_150k_12_230331.csv'
with open(os.path.split(CSV_URL)[1], 'wb') as f, \
        requests.get(CSV_URL, stream=True) as r:
    for line in r.iter_lines():
        f.write(line+'\n'.encode())


<h1>Download All the Files of PPP Loan</h1>

In [2]:
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient
import os
import requests
from bs4 import BeautifulSoup
import re
import boto3
import glob
import pandas as pd
import yfinance as yf

In [14]:

url = "https://data.sba.gov/dataset/ppp-foia"
# Download the webpage content
response = requests.get(url)
page_content = response.text
print(f"Downloaded {csv_filename} to {destination_file}.")
print("All CSV files have been downloaded.")

Downloaded public_150k_plus_230630.csv to data/ppploan\public_150k_plus_230630.csv.
All CSV files have been downloaded.


In [37]:
# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(page_content, "html.parser")
# Find all anchor tags containing CSV file links
csv_links = soup.find_all("a", href=re.compile(".*\.csv"))
# Define the destination folder for the downloaded CSV files
destination_folder = "./data/ppploan"

In [40]:
# Download each CSV file to the specified destination folder
#for link in csv_links:
    csv_url = link["href"]
    csv_filename = os.path.basename(csv_url)
    destination_file = os.path.join(destination_folder, csv_filename)
    print(f"Downloading {csv_url}...")
    # Download the CSV file
    with requests.get(csv_url, stream=True) as r:
        r.raise_for_status()
        with open(destination_file, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

In [ ]:
C:\Users\aladd\Downloads\Python Data Analysis\CIS 4400\data\ppploan\merged

<h1> Upload All the PPP Loan Files to Azure</h1>

In [166]:
folders = ["data/ppploan/merged"]

# Azure Blob Storage Configuration
azure_connection_string = 'DefaultEndpointsProtocol=https;AccountName=sapppcis4400dev;AccountKey=87qq/ubn/GtI/KxMzWUfxVvEu5hnWhOSa9FSBmYq02i1HVKDOKD13Yc28Vh512tBDqR2GsThKf7c+AStZw5HOg==;EndpointSuffix=core.windows.net'

azure_container_name = 'sapppcis4400dev'



In [167]:
# Upload files to Azure Blob Storage

def upload_to_azure(source_file, destination_path):
    blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)
    container_client = blob_service_client.get_container_client(azure_container_name)
    blob_client = container_client.get_blob_client(destination_path)
    with open(source_file, "rb") as data:
        blob_client.upload_blob(data)


In [168]:
# Iterate through each folder, and upload the CSV files to S3, GCS, and Azure

for folder in folders:
    csv_files = glob.glob(os.path.join(folder, "*.csv"))
    for csv_file in csv_files:
        destination_path = os.path.join("uploaded", os.path.basename(csv_file))
        # Upload to Azure Blob Storage
        upload_to_azure(csv_file, destination_path)
        print(f"Uploaded {csv_file} to S3, GCS, and Azure.")

KeyboardInterrupt: 

<h1> Merged All PPP Loan Files</h1>

In [2]:
path = '.\data\ppploan'

filenames = glob.glob(path + '\*.csv')

In [8]:
# combine all the ppp loand files
df = pd.DataFrame()
for file in filenames:
    temp = pd.read_csv(file)
    df = df.append(temp, ignore_index=True)
   
    

C:\Users\aladd\AppData\Local\Temp\ipykernel_11996\2546215952.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp)
C:\Users\aladd\AppData\Local\Temp\ipykernel_11996\2546215952.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp)
C:\Users\aladd\AppData\Local\Temp\ipykernel_11996\2546215952.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp)
C:\Users\aladd\AppData\Local\Temp\ipykernel_11996\2546215952.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp)
C:\Users\aladd\AppData\Local\Temp\ipykernel_11996\2546215952.py:4: FutureWarning: The frame.append method is

In [10]:
df.to_csv("./data/ppploan/merged.csv")

In [11]:
list(df.columns)

['LoanNumber',
 'DateApproved',
 'SBAOfficeCode',
 'ProcessingMethod',
 'BorrowerName',
 'BorrowerAddress',
 'BorrowerCity',
 'BorrowerState',
 'BorrowerZip',
 'LoanStatusDate',
 'LoanStatus',
 'Term',
 'SBAGuarantyPercentage',
 'InitialApprovalAmount',
 'CurrentApprovalAmount',
 'UndisbursedAmount',
 'FranchiseName',
 'ServicingLenderLocationID',
 'ServicingLenderName',
 'ServicingLenderAddress',
 'ServicingLenderCity',
 'ServicingLenderState',
 'ServicingLenderZip',
 'RuralUrbanIndicator',
 'HubzoneIndicator',
 'LMIIndicator',
 'BusinessAgeDescription',
 'ProjectCity',
 'ProjectCountyName',
 'ProjectState',
 'ProjectZip',
 'CD',
 'JobsReported',
 'NAICSCode',
 'Race',
 'Ethnicity',
 'UTILITIES_PROCEED',
 'PAYROLL_PROCEED',
 'MORTGAGE_INTEREST_PROCEED',
 'RENT_PROCEED',
 'REFINANCE_EIDL_PROCEED',
 'HEALTH_CARE_PROCEED',
 'DEBT_INTEREST_PROCEED',
 'BusinessType',
 'OriginatingLenderLocationID',
 'OriginatingLender',
 'OriginatingLenderCity',
 'OriginatingLenderState',
 'Gender',
 'Vete

<h3>Manually scrape symbols for 4000 companies</h3>

In [104]:
# get company stock ticker

def getTicker(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()
    
    if len(data['quotes']) == 0:
        company_code = "0"
    else:
        company_code = data['quotes'][0]['symbol'] 

    return company_code

In [121]:
# divided the company names into 20 arrays
company_name = np.array_split(state_loan['name'].unique(), 20)

In [165]:
company_name_df1 = pd.DataFrame({'name': company_name[1],
                                'symbol':[getTicker(state) for state in company_name[1]]})

In [129]:
company_name_df2 = pd.DataFrame({'name': company_name[2],
                                'symbol':[getTicker(state) for state in company_name[2]]})

In [130]:
company_name_df3 = pd.DataFrame({'name': company_name[3],
                                'symbol':[getTicker(state) for state in company_name[3]]})

In [133]:
company_name_df4 = pd.DataFrame({'name': company_name[4],
                                'symbol':[getTicker(state) for state in company_name[4]]})

In [134]:
company_name_df5 = pd.DataFrame({'name': company_name[5],
                                'symbol':[getTicker(state) for state in company_name[5]]})

In [135]:
company_name_df6 = pd.DataFrame({'name': company_name[6],
                                'symbol':[getTicker(state) for state in company_name[6]]})

In [136]:
company_name_df7 = pd.DataFrame({'name': company_name[7],
                                'symbol':[getTicker(state) for state in company_name[7]]})

In [148]:
company_name_df8 = pd.DataFrame({'name': company_name[8],
                                'symbol':[getTicker(state) for state in company_name[8]]})

In [149]:
company_name_df9 = pd.DataFrame({'name': company_name[9],
                                'symbol':[getTicker(state) for state in company_name[9]]})

In [151]:
company_name_df10 = pd.DataFrame({'name': company_name[10],
                                'symbol':[getTicker(state) for state in company_name[10]]})

In [152]:
company_name_df11 = pd.DataFrame({'name': company_name[11],
                                'symbol':[getTicker(state) for state in company_name[11]]})

In [153]:
company_name_df12 = pd.DataFrame({'name': company_name[12],
                                'symbol':[getTicker(state) for state in company_name[12]]})

In [154]:
company_name_df13 = pd.DataFrame({'name': company_name[13],
                                'symbol':[getTicker(state) for state in company_name[13]]})

In [158]:
company_name_df14 = pd.DataFrame({'name': company_name[14],
                                'symbol':[getTicker(state) for state in company_name[14]]})

In [159]:
company_name_df15 = pd.DataFrame({'name': company_name[15],
                                'symbol':[getTicker(state) for state in company_name[15]]})

In [160]:
company_name_df16 = pd.DataFrame({'name': company_name[16],
                                'symbol':[getTicker(state) for state in company_name[16]]})

In [161]:
company_name_df17 = pd.DataFrame({'name': company_name[17],
                                'symbol':[getTicker(state) for state in company_name[17]]})

In [162]:
company_name_df18 = pd.DataFrame({'name': company_name[18],
                                'symbol':[getTicker(state) for state in company_name[18]]})

In [163]:
company_name_df19 = pd.DataFrame({'name': company_name[19],
                                'symbol':[getTicker(state) for state in company_name[19]]})

In [164]:
company_name_df0 = pd.DataFrame({'name': company_name[0],
                                'symbol':[getTicker(state) for state in company_name[0]]})

In [231]:
df = pd.DataFrame()

In [272]:

df = df.append(company_name_df0)

C:\Users\aladd\AppData\Local\Temp\ipykernel_22256\3618105679.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(company_name_df0)


In [277]:
df.to_csv("./data/ppploan/companyName_symbol_loan.csv")

In [284]:
df

,name,symbol
0,b1BANK,0
1,Gulf Coast Bank and Trust Company,0
2,Cornerstone National Bank & Trust Company,0
3,"Hinsdale Bank & Trust Company, National Associ...",ZION
4,Southern Bank and Trust Company,0
...,...,...
227,Montecito Bank & Trust,0
228,Salem Five Cents Savings Bank,0
229,First International Bank & Trust,0
230,The Cape Cod Five Cents Savings Bank,0


In [286]:
state_loan_symbol = pd.merge(state_loan, df, how='outer', on='name')

In [287]:
state_loan_symbol.to_csv("./data/ppploan/state_loan_symbol.csv")

In [291]:
# Top 5 Bank per State
Top5Bank = pd.DataFrame()

for state in state_loan["state"].unique():
    Top5Bank = df.append(state_loan_symbol[state_loan_symbol.state == state].head(5))


C:\Users\aladd\AppData\Local\Temp\ipykernel_22256\2182463014.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Top5Bank = df.append(state_loan_symbol[state_loan_symbol.state == state].head(5))
C:\Users\aladd\AppData\Local\Temp\ipykernel_22256\2182463014.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Top5Bank = df.append(state_loan_symbol[state_loan_symbol.state == state].head(5))
C:\Users\aladd\AppData\Local\Temp\ipykernel_22256\2182463014.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Top5Bank = df.append(state_loan_symbol[state_loan_symbol.state == state].head(5))
C:\Users\aladd\AppData\Local\Temp\ipykernel_22256\2182463014.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pan

In [293]:
# Write the top 5 bank per state dataset as csv file
Top5Bank.to_csv("./data/ppploan/Top5BankPerState_loan.csv")

In [292]:
# if symbol is 0 means that the company does not have a symbol in yahoo.com
Top5Bank

,name,state,amount,count,symbol
0,"JPMorgan Chase Bank, National Association",NY,1.108796e+10,102492,JPM-PD
56,"Bank of America, National Association",NY,2.726851e+09,30684,BAC
132,"U.S. Bank, National Association",NY,3.597424e+07,237,IUS2.DE
175,"Wells Fargo Bank, National Association",NY,2.169126e+08,3685,AAADAXX
214,Manufacturers and Traders Trust Company,NY,4.998866e+09,31756,0
...,...,...,...,...,...
3022,"Prestamos CDFI, LLC",AE,2.000000e+04,1,BTM
18570,Navy FCU,AE,4.052500e+04,2,0
53,"JPMorgan Chase Bank, National Association",AE,3.462000e+03,1,JPM-PD
3022,"Prestamos CDFI, LLC",AE,2.000000e+04,1,BTM


In [4]:
merged_df = pd.read_csv('.\data\ppploan\merged\merged.csv')

C:\Users\aladd\AppData\Local\Temp\ipykernel_21252\851823603.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('.\data\ppploan\merged\merged.csv')


In [5]:
list(merged_df.columns)

['Unnamed: 0',
 'LoanNumber',
 'DateApproved',
 'SBAOfficeCode',
 'ProcessingMethod',
 'BorrowerName',
 'BorrowerAddress',
 'BorrowerCity',
 'BorrowerState',
 'BorrowerZip',
 'LoanStatusDate',
 'LoanStatus',
 'Term',
 'SBAGuarantyPercentage',
 'InitialApprovalAmount',
 'CurrentApprovalAmount',
 'UndisbursedAmount',
 'FranchiseName',
 'ServicingLenderLocationID',
 'ServicingLenderName',
 'ServicingLenderAddress',
 'ServicingLenderCity',
 'ServicingLenderState',
 'ServicingLenderZip',
 'RuralUrbanIndicator',
 'HubzoneIndicator',
 'LMIIndicator',
 'BusinessAgeDescription',
 'ProjectCity',
 'ProjectCountyName',
 'ProjectState',
 'ProjectZip',
 'CD',
 'JobsReported',
 'NAICSCode',
 'Race',
 'Ethnicity',
 'UTILITIES_PROCEED',
 'PAYROLL_PROCEED',
 'MORTGAGE_INTEREST_PROCEED',
 'RENT_PROCEED',
 'REFINANCE_EIDL_PROCEED',
 'HEALTH_CARE_PROCEED',
 'DEBT_INTEREST_PROCEED',
 'BusinessType',
 'OriginatingLenderLocationID',
 'OriginatingLender',
 'OriginatingLenderCity',
 'OriginatingLenderState',
 '

In [5]:
#Fact Table Primary key
merged_df['ppploanid'] = range(1, len(merged_df) + 1)

In [11]:
# Create Borrower Dimension 

dim_borrower_df = merged_df[["BorrowerName","BorrowerAddress",
"BorrowerCity","BorrowerState","BorrowerZip"]]

In [77]:
# Create a mapping from borrowername to a unique ID 
unique_ids = pd.factorize(dim_borrower_df['BorrowerName'])[0]
dim_borrower_df['idborrower'] = unique_ids
unique_ids = None
dim_borrower_df.drop_duplicates(subset=['idborrower'],inplace=True)

C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\1807699423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_borrower_df['idborrower'] = unique_ids
C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\1807699423.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_borrower_df.drop_duplicates(subset=['idborrower'],inplace=True)


In [78]:
dim_borrower_df

,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,idborrower
0,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150-9662,0
1,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420-9000,1
2,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112-2922,2
3,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407-1740,3
4,AERO BOX LLC,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...
11462040,OMTARA MUSIC & EVENTS LLC,5100 NE Going St.,Portland,OR,97218,8788502
11462041,BEST COAST SHOWCASE,4704 TIE ST,SALEM,OR,97305-3583,8788503
11462042,CRESCENT HOMES INC,2925 MILES WAY,EUGENE,OR,97404-2395,8788504
11462043,"APRIL SEVERSON EVENTS, LLC",1716 SE 24TH AVE,PORTLAND,OR,97214-4858,8788505


In [79]:
dim_borrower_df.to_csv("./data/ppploan/dimension_fact_tables/dim_borrower.csv")

In [13]:
# Create OriginatingLender

dim_originationglender_df = merged_df[["OriginatingLender", 
"OriginatingLenderCity","OriginatingLenderState"]]

In [14]:
# Create a mapping from originatinglender to a unique ID 
unique_ids = pd.factorize(dim_originationglender_df['OriginatingLender'])[0]
dim_originationglender_df['idoriginatinglender'] = unique_ids
unique_ids = None

C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\68934753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_originationglender_df['idoriginatinglender'] = unique_ids


In [74]:
dim_originationglender_df.drop_duplicates(subset=['idoriginatinglender'],inplace=True)

C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\1353949270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_originationglender_df.drop_duplicates(subset=['idoriginatinglender'],inplace=True)


In [76]:
dim_originationglender_df.to_csv("./data/ppploan/dimension_fact_tables/dim_originationglender.csv")


In [51]:
# Create ServicicingLender 

dim_servicinglender_df = merged_df[["ServicingLenderName",
"ServicingLenderAddress","ServicingLenderCity","ServicingLenderState","ServicingLenderZip" ]]

In [52]:
dim_servicinglender_df

,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip
0,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429
1,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429
2,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024
3,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024
4,The Huntington National Bank,17 S High St.,COLUMBUS,OH,43215-3413
...,...,...,...,...,...
11462040,Beneficial State Bank,"1438 Webster St, Ste 100",OAKLAND,CA,94612-3229
11462041,Marion & Polk Schools CU,451 Division St NE,SALEM,OR,97301-2569
11462042,Umpqua Bank,"445 SE Main St, First Fl",ROSEBURG,OR,97470-4934
11462043,Umpqua Bank,"445 SE Main St, First Fl",ROSEBURG,OR,97470-4934


In [63]:
# replace 'JPMorgan Chase Bank, National Association' to "JPMorgan Chase & Co."
# replace 'Wells Fargo Bank, National Association' to  "Wells Fargo & Company"
# replace "Signature Bank",  "Signature Bank of Arkansas' ,'Signature Bank, National Association' to "Signature Bank"


dim_servicinglender_df.loc[dim_servicinglender_df.ServicingLenderName == 'JPMorgan Chase Bank, National Association', 
                           'ServicingLenderName'] = "JPMorgan Chase & Co." 
dim_servicinglender_df.loc[dim_servicinglender_df.ServicingLenderName == 'Wells Fargo Bank, National Association', 
                           'ServicingLenderName'] = "Wells Fargo & Company" 
dim_servicinglender_df.loc[dim_servicinglender_df.ServicingLenderName == 'Signature Bank of Arkansas', 
                           'ServicingLenderName'] = "Signature Bank" 
dim_servicinglender_df.loc[dim_servicinglender_df.ServicingLenderName == 'Signature Bank, National Association', 
                           'ServicingLenderName'] = "Signature Bank" 

C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\4104573197.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_servicinglender_df.loc[dim_servicinglender_df.ServicingLenderName == 'JPMorgan Chase Bank, National Association',
C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\4104573197.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_servicinglender_df.loc[dim_servicinglender_df.ServicingLenderName == 'Wells Fargo Bank, National Association',
C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\4104573197.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the

In [65]:
dim_servicinglender_df

,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip
0,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429
1,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429
2,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024
3,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024
4,The Huntington National Bank,17 S High St.,COLUMBUS,OH,43215-3413
...,...,...,...,...,...
11462040,Beneficial State Bank,"1438 Webster St, Ste 100",OAKLAND,CA,94612-3229
11462041,Marion & Polk Schools CU,451 Division St NE,SALEM,OR,97301-2569
11462042,Umpqua Bank,"445 SE Main St, First Fl",ROSEBURG,OR,97470-4934
11462043,Umpqua Bank,"445 SE Main St, First Fl",ROSEBURG,OR,97470-4934


In [66]:
# Create a mapping from servicinglender to a unique ID 
unique_ids = pd.factorize(dim_servicinglender_df['ServicingLenderName'])[0]
dim_servicinglender_df['idservicinglender'] = unique_ids
unique_ids = None

C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\1907491759.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_servicinglender_df['idservicinglender'] = unique_ids


In [70]:
dim_servicinglender_df.drop_duplicates(subset=['idservicinglender'],inplace=True)

C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\1048607126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_servicinglender_df.drop_duplicates(subset=['idservicinglender'],inplace=True)


In [71]:
dim_servicinglender_df

,ServicingLenderName,ServicingLenderAddress,ServicingLenderCity,ServicingLenderState,ServicingLenderZip,idservicinglender
0,Synovus Bank,1148 Broadway,COLUMBUS,GA,31901-2429,0
2,"Bank of America, National Association","100 N Tryon St, Ste 170",CHARLOTTE,NC,28202-4024,1
4,The Huntington National Bank,17 S High St.,COLUMBUS,OH,43215-3413,2
13,Northrim Bank,3111 'C' St,ANCHORAGE,AK,99503,3
14,"National Cooperative Bank, National Association",139 S High St,HILLSBORO,OH,45133-1442,4
...,...,...,...,...,...,...
11262293,Tulsa Economic Development Corporation,125 West 3rd Street,Tulsa,OK,74103,4630
11262990,Community State Bank of Canton,101 W Main St,CANTON,OK,73724,4631
11279778,The First National Bank of Fletcher,401 W Cole,FLETCHER,OK,73541,4632
11321319,Muskogee FCU,1910 W Shawnee,MUSKOGEE,OK,74401-2299,4633


In [72]:
dim_servicinglender_df.to_csv("./data/ppploan/dimension_fact_tables/dim_servicinglender.csv")


In [32]:

# Create NAICS 

naics_df = pd.read_csv("./data/ppploan/dimension_fact_tables/naics.csv")
naics_df = naics_df.drop(naics_df.columns[-1], axis=1)
naics_df.columns = ["NAICSCode","Description"]
naics_df = naics_df.drop(naics_df.index[0])
naics_df.head()
naics_df.to_csv("./data/ppploan/dimension_fact_tables/dim_naics.csv")

In [34]:

# Create Fact 

fact_df = merged_df[["ppploanid","NAICSCode",
"SBAOfficeCode","DateApproved","SBAGuarantyPercentage",
"ProcessingMethod","Term","LoanStatusDate","LoanStatus","UndisbursedAmount",
"InitialApprovalAmount","CurrentApprovalAmount","ForgivenessAmount","ForgivenessDate",
"PAYROLL_PROCEED" ]]
fact_df["idborrower"] = dim_borrower_df['idborrower']
fact_df["idservicinglender"] = dim_servicinglender_df['idservicinglender']
fact_df['idoriginatinglender'] =dim_originationglender_df['idoriginatinglender']


C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\505799051.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_df["idborrower"] = dim_borrower_df['idborrower']
C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\505799051.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_df["idservicinglender"] = dim_servicinglender_df['idservicinglender']
C:\Users\aladd\AppData\Local\Temp\ipykernel_15480\505799051.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [35]:

fact_df.head()

,ppploanid,NAICSCode,SBAOfficeCode,DateApproved,SBAGuarantyPercentage,ProcessingMethod,Term,LoanStatusDate,LoanStatus,UndisbursedAmount,InitialApprovalAmount,CurrentApprovalAmount,ForgivenessAmount,ForgivenessDate,PAYROLL_PROCEED,idborrower,idservicinglender,idoriginatinglender
0,1,325510.0,464.0,05/01/2020,100,PPP,24,12/18/2020,Paid in Full,0.0,769358.78,769358.78,773553.37,11/20/2020,769358.78,0,0,0
1,2,561730.0,464.0,05/01/2020,100,PPP,24,09/28/2021,Paid in Full,0.0,736927.79,736927.79,746336.24,08/12/2021,736927.79,1,0,0
2,3,NaN,1013.0,05/01/2020,100,PPP,24,03/17/2021,Paid in Full,0.0,691355.00,691355.00,696677.49,02/10/2021,691355.00,2,0,1
3,4,236115.0,920.0,05/01/2020,100,PPP,24,10/16/2021,Paid in Full,0.0,499871.00,499871.00,395264.11,09/10/2021,499871.00,3,0,1
4,5,484210.0,101.0,05/01/2020,100,PPP,24,08/17/2021,Paid in Full,0.0,367437.00,367437.00,370819.35,04/08/2021,367437.00,4,0,2


In [36]:

fact_df.to_csv("./data/ppploan/dimension_fact_tables/fact_ppp.csv")

In [5]:
#Fact Table Primary key
#merged_df['ppploanid'] = range(1, len(merged_df) + 1)

# Create a mapping from borrowername to a unique ID 
#unique_ids = pd.factorize(merged_df['BorrowerName'])[0]
#merged_df['idborrower'] = unique_ids
#unique_ids = None

# Create a mapping from servicinglender to a unique ID 
#unique_ids = pd.factorize(merged_df['ServicingLenderName'])[0]
#merged_df['idservicinglender'] = unique_ids
#unique_ids = None


# Create a mapping from originatinglender to a unique ID 
#unique_ids = pd.factorize(merged_df['OriginatingLender'])[0]
#merged_df['idoriginatinglender'] = unique_ids
#unique_ids = None

# Create a mapping from originatinglender to a unique ID 
#merged_df['DateApproved'] = pd.to_datetime(merged_df['DateApproved'])
#merged_df['LoanStatusDate'] = pd.to_datetime(merged_df['LoanStatusDate'])
#merged_df['ForgivenessDate'] = pd.to_datetime(merged_df['ForgivenessDate'])

# Create a mapping for dateid
#merged_df['idDateApproved'] = merged_df['DateApproved'].dt.strftime('%Y%m%d')
#merged_df['idLoanStatusDate'] = merged_df['LoanStatusDate'].dt.strftime('%Y%m%d')
#merged_df['idForgivenessDate'] = merged_df['ForgivenessDate'].dt.strftime('%Y%m%d')

<h1> More Transformation </h1>

In [10]:
# select columns from merged_df
state_loan = merged_df[['BorrowerCity',
 'BorrowerState', 'CurrentApprovalAmount',
 'ServicingLenderName']]

In [11]:
# replace 'JPMorgan Chase Bank, National Association' to "JPMorgan Chase & Co."
# replace 'Wells Fargo Bank, National Association' to  "Wells Fargo & Company"
# replace "Signature Bank",  "Signature Bank of Arkansas' ,'Signature Bank, National Association' to "Signature Bank"


state_loan.loc[state_loan.ServicingLenderName == 'JPMorgan Chase Bank, National Association', 
                           'ServicingLenderName'] = "JPMorgan Chase & Co." 
state_loan.loc[state_loan.ServicingLenderName == 'Wells Fargo Bank, National Association', 
                           'ServicingLenderName'] = "Wells Fargo & Company" 
state_loan.loc[state_loan.ServicingLenderName == 'Signature Bank of Arkansas', 
                           'ServicingLenderName'] = "Signature Bank" 
state_loan.loc[state_loan.ServicingLenderName == 'Signature Bank, National Association', 
                           'ServicingLenderName'] = "Signature Bank" 

In [8]:
dim_servicinglender_df = pd.read_csv("./data/ppploan/dimension_fact_tables/dim_servicinglender.csv")


In [16]:
state_loan = pd.merge(state_loan, dim_servicinglender_df, on = 'ServicingLenderName' )

In [18]:
state_loan[['BorrowerCity',
 'BorrowerState', 'CurrentApprovalAmount',
 'ServicingLenderName','idservicinglender']]

,BorrowerCity,BorrowerState,CurrentApprovalAmount,ServicingLenderName,idservicinglender
0,Sumter,NaN,769358.78,Synovus Bank,0
1,North Charleston,NaN,736927.79,Synovus Bank,0
2,Saint Johns,NaN,289765.00,Synovus Bank,0
3,Dillingham,AK,288300.43,Synovus Bank,0
4,Dillingham,AK,156767.42,Synovus Bank,0
...,...,...,...,...,...
11462040,Oktaha,OK,3815.20,Muskogee FCU,4633
11462041,Muskogee,OK,3071.45,Muskogee FCU,4633
11462042,Muskogee,OK,2256.87,Muskogee FCU,4633
11462043,Hulbert,OK,505.20,Muskogee FCU,4633


In [20]:
state_loan.to_csv("./data/ppploan/tranformedTables/state_loan.csv")

In [26]:
state_loan_sum_count = state_loan.groupby(['ServicingLenderName','idservicinglender','BorrowerState'])['CurrentApprovalAmount'].aggregate(['sum','count'])

In [27]:
state_loan_sum_count

sum  \
ServicingLenderName                             idservicinglender BorrowerState               
\tFarm Credit Services of Western Arkansas, ACA 4258              AR             9978917.33   
                                                                  OK               73742.05   
                                                                  TX               34834.98   
                                                                  WI               20833.33   
\tFarm Credit of Western Oklahoma, ACA          4256              KS              427582.99   
...                                                                                     ...   
immito, LLC                                     404               VA              104638.00   
                                                                  WA             1216100.00   
                                                                  WI             1627700.00   
                                                                  WV                6200.00   
the Farmers State Bank of Bucklin, Kansas       2470              KS             2638710.70   

                                                                                 count  
ServicingLenderName                             idservicinglender BorrowerState         
\tFarm Credit Services of Western Arkansas, ACA 4258              AR               666  
                                                                  OK                 4  
                                                                  TX                 3  
                                                                  WI                 1  
\tFarm Credit of Western Oklahoma, ACA          4256              KS                21  
...                                                                                ...  
immito, LLC                                     404               VA                 4  
                                                                  WA                 1  
                                                                  WI                 4  
                                                                  WV                 1  
the Farmers State Bank of Bucklin, Kansas       2470              KS               131  

[39170 rows x 2 columns]

In [30]:
# create a dataframe for state loan
state_loan_df = pd.DataFrame(state_loan_sum_count.iloc[:,0].index.tolist(), columns = ["ServicingLenderName","idservicinglender","BorrowerState"])
state_loan_df["CurrentApprovalAmount_Tatal"] = state_loan_sum_count.iloc[:,0].tolist()
state_loan_df["CurrentApprovalAmount_Count"] = state_loan_sum_count.iloc[:,1].tolist()

state_loan_transformed = state_loan_df.sort_values(by = ["CurrentApprovalAmount_Tatal"],ascending = False)

In [48]:
companyName_symbol_loan = pd.read_csv("./data/ppploan/companyName_symbol_loan.csv")

In [49]:
# add replaced company to companyName_symbol_loan

add_symbol = {'name': ["JPMorgan Chase & Co.",
                     'Wells Fargo Bank, National Association',
                     'Signature Bank of Arkansas',
                     'Signature Bank, National Association'],
             'symbol': ['JPM',
                      'WFC',
                      'SBNY',
                      'SBNY']}
companyName_symbol_loan  = companyName_symbol_loan.append(add_symbol,ignore_index=True)

C:\Users\aladd\AppData\Local\Temp\ipykernel_21252\1257875337.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  companyName_symbol_loan  = companyName_symbol_loan.append(add_symbol,ignore_index=True)


In [52]:

companyName_symbol_loan = companyName_symbol_loan.iloc[: , -2:]
companyName_symbol_loan.columns = ["ServicingLenderName", "Symbol"]

In [7]:
#companyName_symbol_loan.to_csv("./data/ppploan/companyName_symbol_loan.csv")


In [8]:
companyName_symbol_loan = pd.read_csv("./data/ppploan/companyName_symbol_loan.csv")
companyName_symbol_loan           

,Unnamed: 0,ServicingLenderName,Symbol
0,0,b1BANK,0
1,1,Gulf Coast Bank and Trust Company,0
2,2,Cornerstone National Bank & Trust Company,0
3,3,"Hinsdale Bank & Trust Company, National Associ...",ZION
4,4,Southern Bank and Trust Company,0
...,...,...,...
4634,4634,First International Bank & Trust,0
4635,4635,The Cape Cod Five Cents Savings Bank,0
4636,4636,First Security Bank,0
4637,4637,"['JPMorgan Chase & Co.', 'Wells Fargo Bank, Na...","['JPM', 'WFC', 'SBNY', 'SBNY']"


In [22]:


state_loan_transformed = pd.read_csv("./data/ppploan/tranformedTables/state_loan_transformed.csv")

In [23]:
state_loan_transformed

,Unnamed: 0,ServicingLenderName,idservicinglender,BorrowerState,CurrentApprovalAmount_Tatal,CurrentApprovalAmount_Count
0,20152,JPMorgan Chase & Co.,60,NY,1.108796e+10,102492
1,3060,"Bank of America, National Association",1,CA,9.517233e+09,133826
2,20122,JPMorgan Chase & Co.,60,CA,8.856228e+09,97447
3,36091,"U.S. Bank, National Association",185,CA,5.830660e+09,68118
4,38137,Wells Fargo & Company,15,CA,5.061066e+09,82603
...,...,...,...,...,...,...
39165,32317,The Bank of Commerce,1267,NM,3.000000e+02,1
39166,17940,Grow Financial FCU,1667,OH,3.000000e+02,1
39167,14853,First National Bank of Hereford,3894,KS,3.000000e+02,1
39168,34823,The State Exchange Bank,2242,IN,2.487500e+02,1


In [24]:
state_loan_transformed_symbol = pd.merge(state_loan_transformed,companyName_symbol_loan, on = 'ServicingLenderName' )[['ServicingLenderName','idservicinglender','BorrowerState','CurrentApprovalAmount_Tatal','CurrentApprovalAmount_Count','Symbol']]

In [26]:
state_loan_transformed_symbol.to_csv("./data/ppploan/tranformedTables/state_loan_transformed_symbol.csv")

In [27]:
state_loan_transformed_symbol

,ServicingLenderName,idservicinglender,BorrowerState,CurrentApprovalAmount_Tatal,CurrentApprovalAmount_Count,Symbol
0,"Bank of America, National Association",1,CA,9.517233e+09,133826,BAC
1,"Bank of America, National Association",1,FL,3.938049e+09,76401,BAC
2,"Bank of America, National Association",1,NY,2.726851e+09,30684,BAC
3,"Bank of America, National Association",1,TX,2.227192e+09,37856,BAC
4,"Bank of America, National Association",1,NJ,1.811579e+09,22777,BAC
...,...,...,...,...,...,...
39058,Mohawk Valley FCU,4611,NY,1.011300e+04,2,0
39059,South Side Community FCU,4462,IL,6.427000e+03,2,0
39060,New Bedford CU,4510,MA,5.824000e+03,1,0
39061,One Vision FCU,4474,IN,5.468750e+03,1,0


In [30]:
# Top 5 Bank per State
Top5Bank = pd.DataFrame()

for state in state_loan_transformed_symbol["BorrowerState"].unique():
    Top5Bank = Top5Bank.append(state_loan_transformed_symbol[state_loan_transformed_symbol.BorrowerState == state].head(5))

C:\Users\aladd\AppData\Local\Temp\ipykernel_15756\2042550817.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Top5Bank = Top5Bank.append(state_loan_transformed_symbol[state_loan_transformed_symbol.BorrowerState == state].head(5))
C:\Users\aladd\AppData\Local\Temp\ipykernel_15756\2042550817.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Top5Bank = Top5Bank.append(state_loan_transformed_symbol[state_loan_transformed_symbol.BorrowerState == state].head(5))
C:\Users\aladd\AppData\Local\Temp\ipykernel_15756\2042550817.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Top5Bank = Top5Bank.append(state_loan_transformed_symbol[state_loan_transformed_symbol.BorrowerState == state].head(5))
C:\Users\aladd\AppData\Local\Temp\

In [32]:
Top5Bank.to_csv("./data/ppploan/tranformedTables/Top5Bank.csv")